In [1]:
import intake
import numpy as np

In [2]:
cat = intake.open_catalog('https://malariagen.github.io/intake/gcs.yml')
cat

gcs:
  args:
    path: https://malariagen.github.io/intake/gcs.yml
  description: ''
  driver: intake.catalog.local.YAMLFileCatalog
  metadata:
    version: 1


In [3]:
ag1 = cat.ag1
list(ag1)

['samples',
 'snps',
 'snps_pass',
 'snps_pass_biallelic',
 'haps',
 'accessibility',
 'allele_counts']

In [4]:
snps = ag1.snps.to_zarr()

In [5]:
chroms = '2R', '2L', '3R', '3L', 'X'

In [6]:
n_snps_wg = 0
for chrom in chroms:
    n_snps_chrom = np.count_nonzero(snps[chrom]['variants/FILTER_PASS'][:])
    print(chrom, n_snps_chrom)
    n_snps_wg += n_snps_chrom
n_snps_wg

2R 14080970
2L 10377280
3R 13167162
3L 9643193
X 5257352


52525957

In [7]:
n_alts_wg = 0
for chrom in chroms:
    filter_pass = snps[chrom]['variants/FILTER_PASS'][:]
    n_alts_chrom = snps[chrom]['variants/numalt'][:][filter_pass].sum()
    print(chrom, n_alts_chrom)
    n_alts_wg += n_alts_chrom
n_alts_wg

2R 17040239
2L 12614918
3R 16420337
3L 12032130
X 6374367


64481991

In [8]:
numalt_bincount_wg = None
for chrom in chroms:
    filter_pass = snps[chrom]['variants/FILTER_PASS'][:]
    numalt_bincount_chrom = np.bincount(snps[chrom]['variants/numalt'][:][filter_pass])
    print(chrom, numalt_bincount_chrom)
    if numalt_bincount_wg is None:
        numalt_bincount_wg = numalt_bincount_chrom
    else:
        numalt_bincount_wg += numalt_bincount_chrom
numalt_bincount_wg

2R [       0 11332702  2537267   211001]
2L [      0 8296600 1923722  156958]
3R [       0 10178803  2723543   264816]
3L [      0 7449486 1998477  195230]
X [      0 4219279  959131   78942]


array([       0, 41476870, 10142140,   906947])

In [9]:
numalt_bincount_wg[2:].sum()

11049087

In [10]:
numalt_bincount_wg[1] / n_snps_wg

0.7896452034181881

In [11]:
numalt_bincount_wg[2:].sum() / n_snps_wg

0.21035479658181191

In [12]:
samples = ag1.samples.read()
samples.head()

/home/aliman/malariagen/binder/conda/envs/dphil-v2.5.1/lib/python3.7/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,index,ox_code,src_code,sra_sample_accession,population,country,region,contributor,contact,year,m_s,sex,n_sequences,mean_coverage,latitude,longitude
0,0,AB0085-C,BF2-4,ERS223996,BFS,Burkina Faso,Pala,Austin Burt,Sam O'Loughlin,2012,S,F,89905852,28.01,11.150,-4.235
1,1,AB0087-C,BF3-3,ERS224013,BFM,Burkina Faso,Bana,Austin Burt,Sam O'Loughlin,2012,M,F,116706234,36.76,11.233,-4.472
2,2,AB0088-C,BF3-5,ERS223991,BFM,Burkina Faso,Bana,Austin Burt,Sam O'Loughlin,2012,M,F,112090460,23.30,11.233,-4.472
3,3,AB0089-C,BF3-8,ERS224031,BFM,Burkina Faso,Bana,Austin Burt,Sam O'Loughlin,2012,M,F,145350454,41.36,11.233,-4.472
4,4,AB0090-C,BF3-10,ERS223936,BFM,Burkina Faso,Bana,Austin Burt,Sam O'Loughlin,2012,M,F,105012254,34.64,11.233,-4.472


In [13]:
allele_counts = ag1.allele_counts_pass.to_zarr()

In [14]:
list(allele_counts)

['2L', '2R', '3L', '3R', 'X']

In [15]:
list(allele_counts['2L'])

['AOM',
 'BFM',
 'BFS',
 'CMS',
 'GAS',
 'GNS',
 'GWA',
 'KES',
 'UGS',
 'all',
 'all_m',
 'all_s']

In [16]:
pops = samples.population.unique()
pops

array(['BFS', 'BFM', 'UGS', 'GWA', 'KES', 'CMS', 'AOM', 'GAS', 'GNS'],
      dtype=object)

In [17]:
acs = dict()
for key in allele_counts['2L']:
    print(key)
    acs[key] = np.concatenate([allele_counts[chrom][key] for chrom in chroms], axis=0)
acs

AOM
BFM
BFS
CMS
GAS
GNS
GWA
KES
UGS
all
all_m
all_s


{'AOM': array([[120,   0,   0,   0],
        [120,   0,   0,   0],
        [120,   0,   0,   0],
        ...,
        [120,   0,   0,   0],
        [120,   0,   0,   0],
        [120,   0,   0,   0]], dtype=int32),
 'BFM': array([[138,   0,   0,   0],
        [138,   0,   0,   0],
        [138,   0,   0,   0],
        ...,
        [132,   0,   0,   0],
        [ 71,  61,   0,   0],
        [132,   0,   0,   0]], dtype=int32),
 'BFS': array([[157,   5,   0,   0],
        [162,   0,   0,   0],
        [162,   0,   0,   0],
        ...,
        [122,   0,   0,   0],
        [122,   0,   0,   0],
        [ 65,  57,   0,   0]], dtype=int32),
 'CMS': array([[522,  28,   0,   0],
        [547,   3,   0,   0],
        [549,   1,   0,   0],
        ...,
        [464,   0,   0,   0],
        [464,   0,   0,   0],
        [256, 208,   0,   0]], dtype=int32),
 'GAS': array([[110,   2,   0,   0],
        [112,   0,   0,   0],
        [112,   0,   0,   0],
        ...,
        [112,   0,   0,   0],


In [18]:
acs['all'].shape

(52525957, 4)

In [19]:
acs['all']

array([[1484,   44,    0,    0],
       [1525,    3,    0,    0],
       [1527,    1,    0,    0],
       ...,
       [1397,    1,    0,    0],
       [1325,   71,    0,    0],
       [ 895,  501,    0,    0]], dtype=int32)

In [20]:
n_singletons = np.count_nonzero(acs['all'][:, 1:] == 1)

In [21]:
n_singletons

21783339

In [22]:
n_pops = np.zeros(acs['all'].shape, dtype=int)
for pop in pops:
    n_pops += acs[pop] > 0
n_pops

array([[9, 5, 0, 0],
       [9, 1, 0, 0],
       [9, 1, 0, 0],
       ...,
       [9, 1, 0, 0],
       [9, 2, 0, 0],
       [9, 7, 0, 0]])

In [23]:
n_pop_private = np.count_nonzero(n_pops[:, 1:] == 1)
n_pop_private

31228996

In [24]:
ac_all_gam = acs['BFS'] + acs['GNS'] + acs['CMS'] + acs['GAS'] + acs['UGS']
ac_all_col = acs['BFM'] + acs['AOM']

In [28]:
n_private_gam = np.count_nonzero((n_pops[:, 1:] > 1) & (ac_all_gam[:, 1:] > 0) & (ac_all_col[:, 1:] == 0))
n_private_gam

17144828

In [29]:
n_private_col = np.count_nonzero((n_pops[:, 1:] > 1) & (ac_all_col[:, 1:] > 0) & (ac_all_gam[:, 1:] == 0))
n_private_col

1252245

In [30]:
n_shared_gam_col = np.count_nonzero((ac_all_gam[:, 1:] > 0) & (ac_all_col[:, 1:] > 0))
n_shared_gam_col

14688252

In [31]:
n_snps_wg

52525957

In [32]:
accessibility = ag1.accessibility.to_zarr()

In [33]:
n_access_wg = 0
for chrom in chroms:
    n_access_chrom = np.count_nonzero(accessibility[chrom]['is_accessible'][:])
    print(chrom, n_access_chrom)
    n_access_wg += n_access_chrom
n_access_wg

2R 40226694
2L 28337781
3R 32878411
3L 25210747
X 14812243


141465876

In [35]:
n_alts_wg / n_access_wg

0.4558130400295263

In [36]:
n_access_wg / n_alts_wg

2.1938819475968105